# Problem Definition

##### Let the non-negative x-coordinate of door offset, and 14 items as follow:

$$\begin{bmatrix} \text{door offset} \\ \text{Billy} \\ \text{Poang} \\ ... \\ \text{Dvala} \end{bmatrix} = \begin{bmatrix} X_{0} \\ X_{1} \\ ... \\ X_{14} \end{bmatrix} $$

##### The objective function needs to be optimized is (15 variables):

$$\begin{align}
\text{argmin}
\sum \limits _{order=1} ^{m} \big(\sum \limits _{i=1} ^{n - 1} \big\lvert{X_{i+1} - X_{i}}\big\rvert + \big\lvert{X_{n} - X_{0}}\big\rvert \big)
\end{align}$$

where:
- $X_{i}$: X-coordinate of item $i$. <br>
- $X_0$: Door offset <br>
- $m$: Total order (in this case study $m = 50$) <br>
- $n$: Article list per order<br>

##### Constraints. Since all items are placed as closely as possible, the constraints can be described:

$\begin{align}
\frac{1}{2}(w_{i} + w_{j}) \leq \big\lvert{X_i - X_j}\big\rvert \leq W - \frac{1}{2}(w_{i} + w_{j}) \newline
\frac{1}{2}w_{i} \leq X_{i} \leq W - \frac{1}{2}w_{i} \newline
\frac{1}{2}min(w_{i}) \leq X_{0} \leq W - \frac{1}{2}min(w_{i})
\end{align}
$

where:
- $w_{i}, w_{j}$: width of item $i$ $\neq$ $j$. <br>
- $W$: Total width of all 14 items <br>
- $min(w_{i})$: Smallest width value <br>

##### Since the result is absolute coordinate, and we want the first item to be at 0. So the output needs to substract the minimal absolute coordinates. Items x-coordinate is:

'Malm': 0.0 <br>
'Dvala': 2.98 <br>
'Ribba': 3.5 <br>
 'Lack': 4.25 <br>
 'Ektorp': 7.1 <br>
 'Billy' and door_offset: 10.9 <br>
 'Fargrik': 12.55 <br>
 'Klippan': 14.3 <br>
 'Stockholm': 17.15 <br>
 'Poang': 18.85 <br>
 'Frakta': 19.6 <br>
 'Docksta': 21.15 <br>
 'Kallax': 23.3 <br>
 'Raskog': 24.75 <br>
 Total Distance  = 1754.1

##### Improvement. The above result is assumed the sequence per order is fixed. But we can actually permute the sequence to optimize the walking distance for each order (>= 2 item in sequence). This can be done at following simple algorithm

FUNCTION NextItem <br>
> Helper Recursive function to find the next nearest item of the given item <br>

FOR each order <br>
  > IF order has at least 2 items THEN <br>
      >> COMPUTE default distance of sequence <br>
      >> FOR item on sequence <br>
          >>> SET item first in the sequence <br>
          >>> CALL NextItem <br>
          >>> COMPUTE Distance of sequence <br>
      >> REPLACE default sequence by sequence with minimal distance <br>

##### order sequence is changed at {1, 2, 5, 6, 9, 10, 12, 21, 22, 26, 27, 29, 32, 34, 37, 38, 45, 46, 48, 50}

##### Total distance = 1440.35

# CODE IMPLEMENTATION

In [1]:
from functions import load_data, write_solution, total_distance, distance
from scipy.optimize import NonlinearConstraint, Bounds, minimize
# Load the problem data
items, item_widths, orders = load_data("data.json")

##### OBJECTIVE FUNCTION

In [2]:
def walking_distance(x: list, 
                     order: list, 
                     item_widths: dict, 
                     items: list) -> float:
    items_dict = dict()
    for variable_order, name_item in enumerate(items):
        items_dict[name_item] = variable_order+1
    order_dist = 0
    for order in orders:
        for i, product in enumerate(order):
            if i == 0:
                # From the door to the first item
                order_dist += abs(x[items_dict[product]] - x[0])
            else:
                # The following items
                prev_product = order[i-1]
                order_dist += abs(x[items_dict[product]] - x[items_dict[prev_product]])
        # Return to the door
        order_dist +=  abs(x[items_dict[product]] - x[0])    
    return order_dist

##### CONSTRAINTS AND BOUNDARY

In [3]:
items_dict = dict()
for variable_order, name_item in enumerate(items):
    items_dict[name_item] = variable_order+1
total_width = sum(item_widths.values())
width_list = list(item_widths.values())
# min and max width value
min_width = min(width_list)
max_width = max(width_list)
constraints_list = []
# Boundary for door offset first
bound_list = [(min_width/2, total_width - min_width/2)] 
for i in range(1, len(item_widths)+1):
    # Pairwise distance between item i, j
    for j in range(i+1, len(item_widths)+1):
        condition = lambda x, i=i, j=j: abs(x[i] - x[j])
        lower_bound = width_list[i-1]/2 + width_list[j-1]/2
        upper_bound = total_width - width_list[i-1] / 2 - width_list[j-1] / 2
        constraints = NonlinearConstraint(condition, lower_bound, upper_bound)
        constraints_list.append(constraints)
    bound_i = (width_list[i-1] / 2, total_width - width_list[i-1] / 2)
    bound_list.append(bound_i)

##### OPTIMIZING

In [5]:
# Define x0 is an iterative process
x0 = [16.32555298, 13.4999999, 21.4499999, 2.5999999, 25.8999999, 19.7499999,
      6.8499999, 9.6999999, 23.7499999, 16.8999999, 15.1499999, 6.0999999,
      22.1999999, 27.3499999, 5.5749999]
res = minimize(walking_distance, 
               x0, 
               args=(orders, item_widths, items), 
               method="SLSQP",
               bounds=tuple(bound_list),
               constraints=constraints_list,
              options={"disp": True,
                       "maxiter": 1000}
              )
if res.success:
    print(res.x)
    door_offset = res.x[0]
    coordinator = {}
    for i, (item, x) in enumerate(items_dict.items()):
        coordinator[item] = res.x[i+1]
    best_sequence = dict(sorted(coordinator.items(), key=lambda item: item[1]))
    metrics = total_distance(item_sequence=list(best_sequence.keys()),
                             door_offset=door_offset,
                             item_widths=item_widths,
                             orders=orders)
else:
    raise("Not convergence. Try again")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1754.1000010886908
            Iterations: 15
            Function evaluations: 283
            Gradient evaluations: 15
[13.50000027 13.5        21.45        2.6        25.9        19.75
  6.85        9.7        23.75       16.9        15.15        6.1
 22.2        27.35        5.575     ]


##### SHIFT THE VALUES SUCH THAT THE FIRST ITEM IS PLACED AT 0

In [6]:
min_coordinate = min(list(best_sequence.values()))
item_coordinate = {}
for k, v in best_sequence.items():
    item_coordinate[k] = v - min_coordinate
door_offset = door_offset - min_coordinate
print("item_coordinate\n", item_coordinate)
print("door_offset", door_offset)

item_coordinate
 {'Malm': 0.0, 'Dvala': 2.9750000000045014, 'Ribba': 3.4999999999958598, 'Lack': 4.249999999993355, 'Ektorp': 7.099999999997147, 'Billy': 10.899999999989912, 'Fargrik': 12.549999999977654, 'Klippan': 14.29999999998146, 'Stockholm': 17.14999999997403, 'Poang': 18.849999999974028, 'Frakta': 19.59999999998141, 'Docksta': 21.149999999973954, 'Kallax': 23.299999999975196, 'Raskog': 24.74999999997148}
door_offset 10.900000272823924


##### EVALUATE TOTAL DISTANCE

In [7]:
walking_distance = total_distance(item_sequence=list(item_coordinate.keys()),
                                 door_offset=door_offset,
                                 item_widths=item_widths,
                                 orders=orders)
print(f"Score = {round(walking_distance, 2)}")

Score = 1754.1


### Now, permute the order sequence to further improve

##### HELPER FUNCTION

In [8]:
# Distance for a order
def order_dist(order: dict, 
               coordinate: dict,
               door_offset: float) -> float:
    
    dist = 0
    for i, product in enumerate(order):
        if i==0:
            # From the door to the first item
            dist += distance(coordinate[order[i]], door_offset)
        else:
            # Distance Between items
            dist += distance(coordinate[order[i]], coordinate[order[i-1]])
    # From the last item back to the door
    dist += distance(coordinate[order[-1]], door_offset)   
    return dist

# Find the nearest item (in the sequence) for a given item
def next_item(initial_item: list,
              rest_item: list,
              item_coordinate: dict) -> None:
    
    if len(rest_item) == 1:
        # Finish and append the last item in the list
        return initial_item.append(rest_item[0])
    # Distance of the rest item respective to the given item (initial_item)
    dist_rest_item = {product: abs(item_coordinate[product] - item_coordinate[initial_item[-1]]) 
                      for product in rest_item}
    # Nearest item with minimal distance to the given item (initial_item)
    nearest_item = min(dist_rest_item, key=dist_rest_item.get)
    # Update initial_item list
    initial_item.append(nearest_item)
    # Update rest_item list
    rest_item = [item for item in order if item not in initial_item]
    return next_item(initial_item,
                     rest_item,
                     item_coordinate)

##### START PERMUTATION

In [9]:
best_orders = []
for k, order in enumerate(orders):
    if len(order) == 1:
        # Do nothing
        best_orders.append(order)
        continue
    # Store all possible sequence distances
    all_distance = {}
    default_dist = order_dist(order, 
                              item_coordinate,
                              door_offset)
    all_distance[" ".join(order)] = default_dist
    for product in order:
        new_order = [product]
        rest_item = [item for item in order if item not in new_order]
        # find the next nearest item in the sequence
        next_item(new_order,
                  rest_item,
                  item_coordinate)
        # Compute new distance for new sequence
        new_dist = order_dist(new_order, 
                              item_coordinate,
                              door_offset)
        new_key = " ".join(new_order)
        if new_key not in all_distance.keys():
            all_distance[new_key] = new_dist
    # Find the minimum distance sequence
    order_change = min(all_distance, key=all_distance.get)
    new_dist = all_distance[order_change]
    order_change = order_change.split(" ")
    if new_dist < 0.99 * default_dist:
        # Find better sequence
        print(f"\nChange at {k}th order")
        print(f"{order}: {default_dist} --->")
        print(f"{order_change}: {new_dist}")
        best_orders.append(order_change)
    else:
        # Nothing change
        best_orders.append(order)


Change at 1th order
['Stockholm', 'Dvala', 'Poang', 'Malm']: 66.04999999988712 --->
['Dvala', 'Malm', 'Stockholm', 'Poang']: 37.699999999948055

Change at 2th order
['Poang', 'Docksta', 'Dvala', 'Ribba', 'Billy', 'Fargrik']: 39.64999945424637 --->
['Dvala', 'Ribba', 'Billy', 'Fargrik', 'Poang', 'Docksta']: 36.3499999999389

Change at 5th order
['Fargrik', 'Ektorp', 'Frakta', 'Docksta']: 31.399999454261074 --->
['Ektorp', 'Fargrik', 'Frakta', 'Docksta']: 28.099999999953614

Change at 6th order
['Frakta', 'Fargrik', 'Poang', 'Raskog', 'Malm']: 63.59999999995047 --->
['Fargrik', 'Poang', 'Frakta', 'Raskog', 'Malm']: 49.49999999994296

Change at 9th order
['Docksta', 'Frakta', 'Klippan', 'Poang', 'Kallax', 'Dvala']: 54.349999999926375 --->
['Dvala', 'Klippan', 'Poang', 'Frakta', 'Docksta', 'Kallax']: 40.649999999941386

Change at 10th order
['Dvala', 'Billy', 'Kallax', 'Lack', 'Ektorp']: 53.950000545602535 --->
['Dvala', 'Lack', 'Ektorp', 'Billy', 'Kallax']: 40.649999999941386

Change at 

##### NEW DISTANCE

In [10]:
best_item_sequence = list(item_coordinate.keys())
best_door_offset = door_offset
new_walking_distance = total_distance(item_sequence=best_item_sequence,
                                     door_offset=best_door_offset,
                                     item_widths=item_widths,
                                     orders=best_orders)
print(f"Score = {round(new_walking_distance, 2)}")

Score = 1440.35
